A Fórmula Mágica é um modelo de fator que classifica as ações por dois fatores:

1. O valor de uma empresa em relação aos seus lucros determina quão “barato” é o preço de mercado de uma ação. Greenblatt define “Barato” como o valor de uma empresa em relação aos seus lucros. Na maioria das vezes, podemos ver isto representado pelo P/L, Greenblatt prefere olhar para EV/EBIT. Isto permite que empresas com diferentes estruturas de endividamento e impostos sejam comparadas mais facilmente. O EV/EBIT compara o Valor da Empresa (Enterprise Value, EV = Capitalização de mercado + Dívida total - Caixa e equivalentes de caixa) ao seu Lucro Antes de Juros e Impostos (Earnings Before Interest and Taxes, EBIT), o EV leva em conta o valor de mercado total da empresa, adicionando as dívidas e subtraindo o caixa, oferecendo assim, uma visão mais abrangente do valor da empresa do que simplesmente seu valor de mercado. O EBIT é uma medida da capacidade de geração de lucro operacional da empresa, excluindo os efeitos das decisões de estrutura de capital e impostos. Um EV/EBIT menor sugere que o preço da ação da empresa pode estar baixo em comparação com a quantidade de lucro operacional que está gerando, indicando que a empresa está subavaliada pelo mercado. No entanto, é importante analisar o EV/EBIT dentro do contexto de cada setor, já que diferentes indústrias têm padrões de avaliação distintos.

2. O retorno sobre o capital determina o quão “boa” é uma empresa. “Bom” é representado pelo ROIC, Greenblatt quantifica a quantidade de capital tangível necessária para operar um negócio e quanto dinheiro cada unidade de capital aplicado irá render. O ROIC é um dos indicadores financeiros mais importantes para analisar a eficiência operacional de uma empresa. Para calcular o ROIC, divide-se o lucro operacional líquido de impostos pelo capital investido total, que inclui dívidas e patrimônio líquido. Um ROIC alto é geralmente interpretado como um sinal de que a empresa está utilizando seu capital de maneira eficaz para gerar lucros.

ROIC utilizada por Greenblatt é diferente da fórmula tradicional

ROIC = EBIT / (Capital Circulante Líquido + Ativos Fixos Líquidos)

Onde o Capital circulante Líquido é uma Proxy do Capital de Giro da Empresa (Ativo Circulante – Passivo Circulante), os Ativos Fixos Líquidos é uma Proxy dos ativos permanentes (imobilizado e intangíveis, utilizados para gerar retorno no longo prazo. A Taxa de Retorno de Lucros para uma empresa é a fórmula tradicional: Taxa de Retorno = Lucro Operacional / Valor da Empresa

Estes dois indicadores são colocados em um ranking cada um e é atribuído o número 1 para a empresa com melhor resultado, número 2 para a empresa com o segundo melhor resultado até o maior número para a pior empresa. Depois somamos os números das duas colunas, e compramos as que tiverem a menor soma.

O Fundamentus nos dá o valor de EV/Ebit enquanto Greenblatt propõe EBIT/EV então bastaria fazer 1/X do valor que aparece lá. Mas vamos usar então a ordem crescente do EV/EBIT

A fim de aderir mais fielmente aos princípios de Greenblatt, empresas pertencentes aos setores de utilidades públicas, bancos e seguradoras devem ser excluídas da seleção.
Essa parte é mais controversa pro Brasil porque tem poucas empresas listadas. 




In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import requests
from IPython.display import display, HTML


def pct_to_float(number):
    """Convert string to float, remove % char and set decimal point to '.'."""
    return float(number.strip("%").replace(".", "").replace(",", "."))


def b_print(df , n=30 , clean=True):
    
    # from IPython.display import display, HTML

    if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
        df['prefixo'] = df['Papel'].astype(str).str[:4]
        df=df.drop_duplicates(subset='prefixo', keep='first')
        # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None

def setor(setor:int):

    url = f'http://www.fundamentus.com.br/resultado.php?setor={setor}'
    hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201' ,
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01' ,
           'Accept-Encoding': 'gzip, deflate' ,
           }
    content = requests.get(url, headers=hdr)
    df = pd.read_html(content.text, decimal=",", thousands='.')[0]
    return list(df['Papel'])



dados do site https://www.fundamentus.com.br/resultado.php

In [2]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
url = 'https://www.fundamentus.com.br/resultado.php'
#junta com a requests
r = requests.get(url, headers=header)
# read_html do pandas põe a tabela num dataframe
funda = pd.read_html(r.text, index_col="Papel",
                     decimal=',', thousands='.',encoding='ISO-8859-1', 
                     converters={'ROE': pct_to_float,
                                 'ROIC': pct_to_float,
                                 'Div.Yield':pct_to_float,
                                 'Mrg Ebit':pct_to_float,
                                 'Mrg. Líq.':pct_to_float,
                                 'Cresc. Rec.5a':pct_to_float,
                                 },
)[0]
data_funds = pd.DataFrame(funda)
data_funds.columns

Index(['Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro',
       'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit',
       'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq',
       'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

In [3]:
earning_yield = 'EV/EBIT'  #Quanto menor melhor
return_on_capital = 'ROIC' #ROIC – Quanto maior melhor

### filtros e ranking

- Empresa precisa ter ROIC positivo
- Empresa precisa ter Lucro Operacional positivo.
- Volume de negociação diário medio maior que R$ 80.000,00 nos últimos 2 meses.

ROIC – Quanto maior melhor

EV/EBIT – Quanto menor melhor

In [4]:
funds = data_funds.copy()

funds =  funds[funds[earning_yield] > 0]
funds =  funds[funds[return_on_capital] > 0]
funds =  funds[funds['Liq.2meses'] > 1000000]     #Volume diário médio negociado nos últimos 2 meses

""" magic formula rank."""
funds["Rank_earnings_yield"]   = funds[earning_yield].rank(ascending=True, method="min")
funds["Rank_return_on_capital"]= funds[return_on_capital].rank(ascending=False, method="min")
funds["Rank_Final"] = (funds["Rank_earnings_yield"] + funds["Rank_return_on_capital"])
funds.sort_values(by="Rank_Final", ascending=True, inplace=True)
funds.reset_index(inplace=True)
funds.index = funds.index + 1

### lista ranqueada

In [5]:
funds.columns

Index(['Papel', 'Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo',
       'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA',
       'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses',
       'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a',
       'Rank_earnings_yield', 'Rank_return_on_capital', 'Rank_Final'],
      dtype='object')

In [6]:
funds[['Papel','EV/EBITDA', 'ROIC', 'Liq.2meses','Rank_earnings_yield', 'Rank_return_on_capital', 'Rank_Final']].head(20)

,Papel,EV/EBITDA,ROIC,Liq.2meses,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
1,PSSA3,0.29,84.27,4.428250e+07,1.0,1.0,2.0
2,WIZC3,1.41,29.67,1.982120e+06,2.0,8.0,10.0
3,PLPL3,3.89,43.68,1.136920e+07,10.0,2.0,12.0
4,CMIN3,3.56,32.77,4.400410e+07,11.0,6.0,17.0
5,PETR4,2.48,23.37,1.181180e+09,5.0,16.0,21.0
6,PETR3,2.61,23.37,3.600050e+08,6.0,16.0,22.0
7,VLID3,3.09,20.98,7.052630e+06,9.0,20.0,29.0
8,VALE3,3.12,19.72,1.275320e+09,8.0,22.0,30.0
9,LREN3,3.42,18.26,2.043610e+08,16.0,25.0,41.0
10,LEVE3,5.56,35.32,1.033680e+07,39.0,3.0,42.0


retira banco, seguradora, utilidade publica (agua e luz - [util-b3](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-utilidade-publica-util-composicao-da-carteira.htm))


In [7]:
bancos = setor(20)
print(*bancos)
seguros = setor(31)
print(*seguros)
util_agua = setor(2)
util_luz = setor(14)
util=util_agua+util_luz
print(*util)


BNBR3 BAZA3 PINE4 BBAS3 BRSR6 PINE3 BRSR3 ABCB4 BMEB3 BMEB4 BRSR5 BMGB4 ITSA3 ITSA4 BEES3 BEES4 ITUB3 BGIP4 BRBI11 BGIP3 BBDC3 SANB3 ITUB4 BPAC5 SANB11 BBDC4 BMIN4 SANB4 BPAC11 BMIN3 MERC4 BPAN4 BPAC3 BSLI4 BSLI3 RPAD5 RPAD6 RPAD3 DMFN3
APER3 WIZC3 PSSA3 BBSE3 IRBR3 CXSE3
AMBP3 SAPR3 SAPR11 SAPR4 CSMG3 SBSP3 ORVR3 ENEV3 AESB3 AURE3 CBEE3 RNEW11 RNEW3 RNEW4 LIGT3 CEED3 CLSC3 CSRN3 CSRN6 NEOE3 CLSC4 EQMA3B CSRN5 CMIG4 CEEB3 TRPL4 CMIG3 COCE5 COCE3 CPFE3 EQPA3 CEBR5 CEBR6 EKTR3 ENGI4 TRPL3 EQPA5 EQPA6 EKTR4 EQPA7 CEBR3 REDE3 EGIE3 ENGI11 EMAE4 GEPA4 TAEE3 TAEE11 TAEE4 GEPA3 ENMT4 CPLE3 ENMT3 ENGI3 CPLE6 ALUP4 ALUP11 ALUP3 AFLT3 CPLE5 EQTL3 SRNA3 ELET3 ELET6 LIPR3 ELET5


#### sem bancos

In [8]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(bancos)],10
)
          

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PSSA3,31.18,7.83,1.61,0.603,4.92,0.452,5.79,0.73,-3.79,0.30,0.29,82.44,7.83,1.15,84.27,20.59,44282500.0,1.250500e+10,0.00,18.04,1.0,1.0,2.0,PSSA
WIZC3,5.99,5.50,1.74,0.792,4.11,0.378,-82.77,1.57,-1.28,1.70,1.41,50.58,24.39,0.98,29.67,31.70,1982120.0,5.490420e+08,0.65,12.98,2.0,8.0,10.0,WIZC
PLPL3,12.68,8.76,3.60,1.082,3.98,1.255,2.18,4.01,6.05,4.01,3.89,26.95,13.24,3.15,43.68,41.10,11369200.0,7.192830e+08,0.53,28.26,10.0,2.0,12.0,PLPL
CMIN3,4.91,7.46,2.23,1.507,10.49,0.906,3.24,4.56,-5.78,4.21,3.56,33.02,20.20,2.79,32.77,29.88,44004100.0,1.208440e+10,0.69,-1.72,11.0,6.0,17.0,CMIN
PETR4,36.51,6.05,1.28,0.954,17.62,0.450,-25.52,2.11,-0.91,3.28,2.48,45.15,15.87,0.90,23.37,21.09,1181180000.0,3.734790e+11,0.89,18.69,5.0,16.0,21.0,PETR
VLID3,19.84,5.01,0.98,0.740,8.39,0.594,1.68,3.75,4.44,3.90,3.09,19.71,14.68,3.12,20.98,19.58,7052630.0,1.653420e+09,0.36,0.93,9.0,20.0,29.0,VLID
VALE3,57.06,5.32,1.26,1.233,15.92,0.538,42.82,3.14,-1.39,3.72,3.12,39.30,23.32,1.08,19.72,23.72,1275320000.0,2.054510e+11,0.41,2.80,8.0,22.0,30.0,VALE
LREN3,15.45,12.90,1.44,1.079,4.26,0.773,3.36,5.27,6.67,4.86,3.42,20.47,8.36,1.66,18.26,11.19,204361000.0,1.031710e+10,0.12,15.83,16.0,25.0,41.0,LREN
LEVE3,33.79,6.01,4.14,1.056,22.78,1.468,5.74,5.65,-107.44,6.26,5.56,18.67,17.57,1.68,35.32,68.84,10336800.0,1.107580e+09,0.82,17.50,39.0,3.0,42.0,LEVE
PRIO3,47.83,7.56,2.40,2.928,0.15,1.144,7.95,5.24,-4.75,6.07,4.64,55.89,38.72,2.04,27.40,31.76,373980000.0,1.776680e+10,0.75,73.92,36.0,10.0,46.0,PRIO


#### sem seguradora

In [9]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(seguros)],10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PLPL3,12.68,8.76,3.60,1.082,3.98,1.255,2.18,4.01,6.05,4.01,3.89,26.95,13.24,3.15,43.68,41.10,11369200.0,7.192830e+08,0.53,28.26,10.0,2.0,12.0,PLPL
CMIN3,4.91,7.46,2.23,1.507,10.49,0.906,3.24,4.56,-5.78,4.21,3.56,33.02,20.20,2.79,32.77,29.88,44004100.0,1.208440e+10,0.69,-1.72,11.0,6.0,17.0,CMIN
PETR4,36.51,6.05,1.28,0.954,17.62,0.450,-25.52,2.11,-0.91,3.28,2.48,45.15,15.87,0.90,23.37,21.09,1181180000.0,3.734790e+11,0.89,18.69,5.0,16.0,21.0,PETR
VLID3,19.84,5.01,0.98,0.740,8.39,0.594,1.68,3.75,4.44,3.90,3.09,19.71,14.68,3.12,20.98,19.58,7052630.0,1.653420e+09,0.36,0.93,9.0,20.0,29.0,VLID
VALE3,57.06,5.32,1.26,1.233,15.92,0.538,42.82,3.14,-1.39,3.72,3.12,39.30,23.32,1.08,19.72,23.72,1275320000.0,2.054510e+11,0.41,2.80,8.0,22.0,30.0,VALE
LREN3,15.45,12.90,1.44,1.079,4.26,0.773,3.36,5.27,6.67,4.86,3.42,20.47,8.36,1.66,18.26,11.19,204361000.0,1.031710e+10,0.12,15.83,16.0,25.0,41.0,LREN
LEVE3,33.79,6.01,4.14,1.056,22.78,1.468,5.74,5.65,-107.44,6.26,5.56,18.67,17.57,1.68,35.32,68.84,10336800.0,1.107580e+09,0.82,17.50,39.0,3.0,42.0,LEVE
PRIO3,47.83,7.56,2.40,2.928,0.15,1.144,7.95,5.24,-4.75,6.07,4.64,55.89,38.72,2.04,27.40,31.76,373980000.0,1.776680e+10,0.75,73.92,36.0,10.0,46.0,PRIO
ASAI3,10.27,20.88,2.88,0.197,0.00,0.315,-4.50,2.77,-0.59,5.29,4.02,7.10,0.94,0.84,17.52,13.78,146241000.0,4.827000e+09,3.67,33.75,22.0,29.0,51.0,ASAI
KEPL3,11.50,7.99,2.80,1.279,8.85,1.361,3.71,7.23,7.82,6.53,5.76,17.69,16.01,2.15,31.43,35.05,11036500.0,7.382410e+08,0.42,31.15,45.0,7.0,52.0,KEPL


#### sem utilidade publica

In [10]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(util)],10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PSSA3,31.18,7.83,1.61,0.603,4.92,0.452,5.79,0.73,-3.79,0.30,0.29,82.44,7.83,1.15,84.27,20.59,44282500.0,1.250500e+10,0.00,18.04,1.0,1.0,2.0,PSSA
WIZC3,5.99,5.50,1.74,0.792,4.11,0.378,-82.77,1.57,-1.28,1.70,1.41,50.58,24.39,0.98,29.67,31.70,1982120.0,5.490420e+08,0.65,12.98,2.0,8.0,10.0,WIZC
PLPL3,12.68,8.76,3.60,1.082,3.98,1.255,2.18,4.01,6.05,4.01,3.89,26.95,13.24,3.15,43.68,41.10,11369200.0,7.192830e+08,0.53,28.26,10.0,2.0,12.0,PLPL
CMIN3,4.91,7.46,2.23,1.507,10.49,0.906,3.24,4.56,-5.78,4.21,3.56,33.02,20.20,2.79,32.77,29.88,44004100.0,1.208440e+10,0.69,-1.72,11.0,6.0,17.0,CMIN
PETR4,36.51,6.05,1.28,0.954,17.62,0.450,-25.52,2.11,-0.91,3.28,2.48,45.15,15.87,0.90,23.37,21.09,1181180000.0,3.734790e+11,0.89,18.69,5.0,16.0,21.0,PETR
VLID3,19.84,5.01,0.98,0.740,8.39,0.594,1.68,3.75,4.44,3.90,3.09,19.71,14.68,3.12,20.98,19.58,7052630.0,1.653420e+09,0.36,0.93,9.0,20.0,29.0,VLID
VALE3,57.06,5.32,1.26,1.233,15.92,0.538,42.82,3.14,-1.39,3.72,3.12,39.30,23.32,1.08,19.72,23.72,1275320000.0,2.054510e+11,0.41,2.80,8.0,22.0,30.0,VALE
LREN3,15.45,12.90,1.44,1.079,4.26,0.773,3.36,5.27,6.67,4.86,3.42,20.47,8.36,1.66,18.26,11.19,204361000.0,1.031710e+10,0.12,15.83,16.0,25.0,41.0,LREN
LEVE3,33.79,6.01,4.14,1.056,22.78,1.468,5.74,5.65,-107.44,6.26,5.56,18.67,17.57,1.68,35.32,68.84,10336800.0,1.107580e+09,0.82,17.50,39.0,3.0,42.0,LEVE
PRIO3,47.83,7.56,2.40,2.928,0.15,1.144,7.95,5.24,-4.75,6.07,4.64,55.89,38.72,2.04,27.40,31.76,373980000.0,1.776680e+10,0.75,73.92,36.0,10.0,46.0,PRIO


#### sem as 3 classes

In [11]:
b_print(
    funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))],
    10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PLPL3,12.68,8.76,3.60,1.082,3.98,1.255,2.18,4.01,6.05,4.01,3.89,26.95,13.24,3.15,43.68,41.10,11369200.0,7.192830e+08,0.53,28.26,10.0,2.0,12.0,PLPL
CMIN3,4.91,7.46,2.23,1.507,10.49,0.906,3.24,4.56,-5.78,4.21,3.56,33.02,20.20,2.79,32.77,29.88,44004100.0,1.208440e+10,0.69,-1.72,11.0,6.0,17.0,CMIN
PETR4,36.51,6.05,1.28,0.954,17.62,0.450,-25.52,2.11,-0.91,3.28,2.48,45.15,15.87,0.90,23.37,21.09,1181180000.0,3.734790e+11,0.89,18.69,5.0,16.0,21.0,PETR
VLID3,19.84,5.01,0.98,0.740,8.39,0.594,1.68,3.75,4.44,3.90,3.09,19.71,14.68,3.12,20.98,19.58,7052630.0,1.653420e+09,0.36,0.93,9.0,20.0,29.0,VLID
VALE3,57.06,5.32,1.26,1.233,15.92,0.538,42.82,3.14,-1.39,3.72,3.12,39.30,23.32,1.08,19.72,23.72,1275320000.0,2.054510e+11,0.41,2.80,8.0,22.0,30.0,VALE
LREN3,15.45,12.90,1.44,1.079,4.26,0.773,3.36,5.27,6.67,4.86,3.42,20.47,8.36,1.66,18.26,11.19,204361000.0,1.031710e+10,0.12,15.83,16.0,25.0,41.0,LREN
LEVE3,33.79,6.01,4.14,1.056,22.78,1.468,5.74,5.65,-107.44,6.26,5.56,18.67,17.57,1.68,35.32,68.84,10336800.0,1.107580e+09,0.82,17.50,39.0,3.0,42.0,LEVE
PRIO3,47.83,7.56,2.40,2.928,0.15,1.144,7.95,5.24,-4.75,6.07,4.64,55.89,38.72,2.04,27.40,31.76,373980000.0,1.776680e+10,0.75,73.92,36.0,10.0,46.0,PRIO
ASAI3,10.27,20.88,2.88,0.197,0.00,0.315,-4.50,2.77,-0.59,5.29,4.02,7.10,0.94,0.84,17.52,13.78,146241000.0,4.827000e+09,3.67,33.75,22.0,29.0,51.0,ASAI
KEPL3,11.50,7.99,2.80,1.279,8.85,1.361,3.71,7.23,7.82,6.53,5.76,17.69,16.01,2.15,31.43,35.05,11036500.0,7.382410e+08,0.42,31.15,45.0,7.0,52.0,KEPL


outros filtros

In [16]:
funds2 =  funds[funds['P/L'] > 0]                # sem prejuizo atual
funds2 =  funds2[(funds['P/L'] < 30)]           # não estar excessivamente cara
funds2 =  funds2[funds['Dív.Brut/ Patrim.'] < 3.5]  # endividamento sob controle
funds2 =  funds2[funds['Cresc. Rec.5a'] > 0]    # crescimento nos ultimos 5 anos

In [17]:
b_print(
    funds2[(~funds2['Papel'].isin(bancos))&(~funds2['Papel'].isin(seguros))&(~funds2['Papel'].isin(util))],
    10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PLPL3,12.68,8.76,3.60,1.082,3.98,1.255,2.18,4.01,6.05,4.01,3.89,26.95,13.24,3.15,43.68,41.10,11369200.0,7.192830e+08,0.53,28.26,10.0,2.0,12.0,PLPL
PETR4,36.51,6.05,1.28,0.954,17.62,0.450,-25.52,2.11,-0.91,3.28,2.48,45.15,15.87,0.90,23.37,21.09,1181180000.0,3.734790e+11,0.89,18.69,5.0,16.0,21.0,PETR
VLID3,19.84,5.01,0.98,0.740,8.39,0.594,1.68,3.75,4.44,3.90,3.09,19.71,14.68,3.12,20.98,19.58,7052630.0,1.653420e+09,0.36,0.93,9.0,20.0,29.0,VLID
VALE3,57.06,5.32,1.26,1.233,15.92,0.538,42.82,3.14,-1.39,3.72,3.12,39.30,23.32,1.08,19.72,23.72,1275320000.0,2.054510e+11,0.41,2.80,8.0,22.0,30.0,VALE
LREN3,15.45,12.90,1.44,1.079,4.26,0.773,3.36,5.27,6.67,4.86,3.42,20.47,8.36,1.66,18.26,11.19,204361000.0,1.031710e+10,0.12,15.83,16.0,25.0,41.0,LREN
LEVE3,33.79,6.01,4.14,1.056,22.78,1.468,5.74,5.65,-107.44,6.26,5.56,18.67,17.57,1.68,35.32,68.84,10336800.0,1.107580e+09,0.82,17.50,39.0,3.0,42.0,LEVE
PRIO3,47.83,7.56,2.40,2.928,0.15,1.144,7.95,5.24,-4.75,6.07,4.64,55.89,38.72,2.04,27.40,31.76,373980000.0,1.776680e+10,0.75,73.92,36.0,10.0,46.0,PRIO
KEPL3,11.50,7.99,2.80,1.279,8.85,1.361,3.71,7.23,7.82,6.53,5.76,17.69,16.01,2.15,31.43,35.05,11036500.0,7.382410e+08,0.42,31.15,45.0,7.0,52.0,KEPL
TGMA3,24.98,7.93,1.86,0.946,9.93,1.300,3.62,6.82,5.91,6.37,5.77,13.87,11.96,3.26,25.96,23.39,5197670.0,8.882300e+08,0.20,12.99,40.0,13.0,53.0,TGMA
GOLL4,1.10,0.24,-0.02,0.025,0.00,0.024,-0.04,0.19,-0.01,5.30,3.11,12.99,10.48,0.32,15.92,-10.06,7440330.0,-1.934800e+10,-0.73,20.37,23.0,36.0,59.0,GOLL


retirar BDR

In [14]:
# Padrão a ser buscado XYZW3[2,3,4,5]
padrao = r'[A-Z]{4}3[2-5]'

# Filtrando as linhas onde o padrão ocorre na coluna 'Papel'
funds[funds['Papel'].str.contains(padrao)]


,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
18,AURA33,16.72,-21.19,2.63,1.447,6.1,0.741,5.81,5.23,-2.05,...,1.55,20.47,-12.42,5573900.0,1.379760e+09,1.35,8.87,40.0,21.0,61.0


A fórmula:
1. Estabeleça uma capitalização de mercado mínima.
2. Exclua ações de serviços públicos e financeiras  (ou seja, fundos mútuos, bancos e companhias de seguros).
3. Excluir empresas estrangeiras (ADRs)
4. Determine o rendimento dos lucros da empresa = EBIT/EV
5. Determine o retorno sobre o capital da empresa = EBIT/ (Ativo Fixo Líquido + Capital de Giro).
6. Classifique todas as empresas acima da capitalização de mercado escolhida pelo maior rendimento de lucros e maior retorno sobre o capital (classificado como porcentagens)
7. Invista em 20 a 30 empresas com melhor classificação, acumulando 2 a 3 posições por mês durante um período de 12 meses
8. Reequilibre o portfólio uma vez por ano, vendendo os perdedores uma semana antes do ano e os vencedores uma semana depois do ano.
9. Continuar por um período de longo prazo (5 a 10+ anos)

https://en.wikipedia.org/wiki/Magic_formula_investing

O retorno sobre o capital de Greenblatt difere de um valor típico de ROE ou ROIC. Dentro da Fórmula Mágica, o retorno sobre o capital de uma empresa é medido como EBIT/capital tangível empregado. Em outras palavras, estamos tentando encontrar os custos tangíveis para o negócio na geração dos lucros reportados dentro do período, onde o capital tangível empregado é definido mais precisamente como Capital Circulante Líquido mais Ativos Fixos Líquidos.

O capital de giro líquido é simplesmente o total do ativo circulante menos o passivo circulante, com um ajuste para remover dívidas com juros de curto prazo do passivo circulante e outro para remover o excesso de caixa. Greenblatt não oferece detalhes sobre como o excesso de caixa deve ser considerado, mas muitas vezes é calculado com base em uma porcentagem do caixa necessário em relação às vendas geradas em um período.

In [15]:
data_funds.loc['INEP4']

Cotação              1.920000e+00
P/L                  1.700000e-01
P/VP                -7.000000e-02
PSR                  1.202000e+01
Div.Yield            0.000000e+00
P/Ativo              7.400000e-02
P/Cap.Giro          -8.000000e-02
P/EBIT              -1.230000e+00
P/Ativ Circ.Liq     -4.000000e-02
EV/EBIT             -1.413000e+01
EV/EBITDA           -1.584000e+01
Mrg Ebit            -9.789500e+02
Mrg. Líq.            6.924440e+03
Liq. Corr.           9.000000e-02
ROIC                -6.370000e+00
ROE                 -4.129000e+01
Liq.2meses           1.112130e+05
Patrim. Líq         -1.182760e+09
Dív.Brut/ Patrim.   -7.600000e-01
Cresc. Rec.5a       -4.420000e+00
Name: INEP4, dtype: float64